## BACKGROUND AND BUSINESS PROBLEM
 
Since mid-2016 to January 31st 2020, the British government was dealing with Brexit and the British withdrawal from the European Union had affected the housing market for both citizens and overseas buyers as the government made it more expensive for landlords and overseas buyers to purchase houses (https://www.bullionvault.com/gold-news/uk-house-prices-022420201). Especially, real estate in London comes at quite a steep price. It is this exact reason why you should not settle and find only the best fit for your needs (https://data.london.gov.uk/housing/housing-market-report/). Therefore, I am going to use the latest data for buyers to make wise and effective decisions.

BUSINESS PROBLEM: What is the wisest investment option for people who seek a house in London now that Brexit is finalised?

SOLUTION: In order to recommend venues and the current average price of real estate, the neighbourhoods of London will be clustered. Recommendations of profitable venues according to amenities and essential facilities such as schools, hospitals & supermarkets will be provided.

## Data

Data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). If the building is divided into flats, there will be a SAON; Street; Locality; Town/City; District; County.

For data visialisation, FourSquare API interface will be used. Then the collected data from HM Land Registry and Foursquare API will be merged and show the most profitable investments in London.

## Methodology

The Methodology for this project will start with 1. Collection of Data
then continue with 2. Exploring and Understanding Data, 3. Data Preparation and Preprocessing 
and 4. Modelling

## 1. Collection of Data

I will start with importing the necessary libraries, then download the data from the HM Land Registry website as follows:

In [1]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /Users/zeynepbokeer/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h0b31af3_1         1.9 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.9 MB

The following packages will be UPDATED:

  openssl                                 1.1.1g-h0b31af3_0 --> 1.1.1g-h0b31af3_1



openssl-1.1.1g       | 1.9 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages already installed.

Libraries imported.


In [6]:
#Reading the data
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

## 2. Explore and Understand Data

I read the dataset that I collected from the HM Land Registry website into a pandas' data frame and checking the raw data by running 'head'

In [9]:
df_ppd.head()

,{7011B109-CFCA-8ED6-E053-6B04A8C075C1},280000,2018-06-04 00:00,IP4 5ES,S,N,F,3,Unnamed: 8,RANDWELL CLOSE,Unnamed: 10,IPSWICH,IPSWICH.1,SUFFOLK,A,A.1
0,{7011B109-CFCB-8ED6-E053-6B04A8C075C1},280000,2018-05-29 00:00,IP1 4BS,T,N,F,261,NaN,NORWICH ROAD,NaN,IPSWICH,IPSWICH,SUFFOLK,A,A
1,{7011B109-CFCC-8ED6-E053-6B04A8C075C1},170000,2018-04-27 00:00,IP4 4BH,T,N,F,31,NaN,PARADE ROAD,NaN,IPSWICH,IPSWICH,SUFFOLK,A,A
2,{7011B109-CFCD-8ED6-E053-6B04A8C075C1},246000,2018-05-25 00:00,IP1 6NB,S,N,F,42,NaN,ELMCROFT ROAD,NaN,IPSWICH,IPSWICH,SUFFOLK,A,A
3,{7011B109-CFCE-8ED6-E053-6B04A8C075C1},180000,2018-06-08 00:00,IP3 9LZ,T,N,F,48,NaN,WYNTERTON CLOSE,NaN,IPSWICH,IPSWICH,SUFFOLK,A,A
4,{7011B109-CFCF-8ED6-E053-6B04A8C075C1},245000,2018-05-11 00:00,IP1 4BU,T,N,F,235,NaN,NORWICH ROAD,NaN,IPSWICH,IPSWICH,SUFFOLK,A,A


Checking the number of rows and columns

In [10]:
df_ppd.shape

(1030277, 16)

As you can see, we have 1030277 columns and 16 rows

## 3. Data preparation and preprocessing

To prepare the dataset for the modeling process, I have to perform below-mentioned steps:

1- Rename the column names

2- Format the date column

3- Sort data by date of sale

4- Select data only for the city of London

5- Make a list of street names in London

6- Calculate the street-wise average price of the property

7- Read the street-wise coordinates into a data frame, eliminating recurring word London from individual names

8- Join the data to find the coordinates of locations which fit into my budget

9- Plot recommended locations on London map along with current market prices

In [11]:
# Rename the column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [12]:
# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2017
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2017].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)


In [13]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in London
streets = df_ppd_london['Street'].unique().tolist()

In [14]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

In [15]:
# Assign an Upper and Lower Limit to my budget
df_affordable = df_grp_price.query("(Avg_Price >= 2000000) & (Avg_Price <= 2200000)")

In [16]:
# Display the dataframe
df_affordable

,Street,Avg_Price
146,AIREDALE AVENUE,2.022500e+06
197,ALBION STREET,2.096667e+06
412,ANTHONY WAY,2.000000e+06
554,ASHCHURCH PARK VILLAS,2.150000e+06
673,AVENUE ROAD,2.143471e+06
762,BALLINGDON ROAD,2.105000e+06
1092,BELSIZE CRESCENT,2.000000e+06
1135,BERESFORD TERRACE,2.100000e+06
1193,BETTRIDGE ROAD,2.025000e+06
1384,BLYTHS WHARF,2.000000e+06


In [47]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [48]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 146
item: Street                AIREDALE AVENUE
Avg_Price                  2.0225e+06
city_coord    (53.4363678, -1.106305)
Name: 146, dtype: object
item.Street only: AIREDALE AVENUE
index: 197
item: Street                     ALBION STREET
Avg_Price                    2.09667e+06
city_coord    (-33.8197594, 151.0102727)
Name: 197, dtype: object
item.Street only: ALBION STREET
index: 412
item: Street                     ANTHONY WAY
Avg_Price                        2e+06
city_coord    (44.094651, -123.151242)
Name: 412, dtype: object
item.Street only: ANTHONY WAY
index: 554
item: Street           ASHCHURCH PARK VILLAS
Avg_Price                     2.15e+06
city_coord    (51.5000507, -0.2421733)
Name: 554, dtype: object
item.Street only: ASHCHURCH PARK VILLAS
index: 673
item: Street                    AVENUE ROAD
Avg_Price                 2.14347e+06
city_coord    (51.4067969, -0.049519)
Name: 673, dtype: object
item.Street only: AVENUE ROAD
index: 762
item: Street                

In [49]:
geolocator = Nominatim()

ConfigurationError: Using Nominatim with default or sample `user_agent` "geopy/2.0.0" is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`.

In [50]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

/Users/zeynepbokeer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [51]:
df_affordable

,Street,Avg_Price,city_coord
146,AIREDALE AVENUE,2.022500e+06,"(53.4363678, -1.106305)"
197,ALBION STREET,2.096667e+06,"(-33.8197594, 151.0102727)"
412,ANTHONY WAY,2.000000e+06,"(44.094651, -123.151242)"
554,ASHCHURCH PARK VILLAS,2.150000e+06,"(51.5000507, -0.2421733)"
673,AVENUE ROAD,2.143471e+06,"(51.4067969, -0.049519)"
762,BALLINGDON ROAD,2.105000e+06,"(51.4541892, -0.1588555)"
1092,BELSIZE CRESCENT,2.000000e+06,"(51.5495675, -0.1727325)"
1135,BERESFORD TERRACE,2.100000e+06,"(55.9757427, -3.2031975)"
1193,BETTRIDGE ROAD,2.025000e+06,"(57.0320376, -2.1476985)"
1384,BLYTHS WHARF,2.000000e+06,"(51.50900105, -0.034583431747228985)"


In [52]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

/Users/zeynepbokeer/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [53]:
df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
146,AIREDALE AVENUE,2.022500e+06,"(53.4363678, -1.106305)",53.436368,-1.106305
197,ALBION STREET,2.096667e+06,"(-33.8197594, 151.0102727)",-33.819759,151.010273
412,ANTHONY WAY,2.000000e+06,"(44.094651, -123.151242)",44.094651,-123.151242
554,ASHCHURCH PARK VILLAS,2.150000e+06,"(51.5000507, -0.2421733)",51.500051,-0.242173
673,AVENUE ROAD,2.143471e+06,"(51.4067969, -0.049519)",51.406797,-0.049519
762,BALLINGDON ROAD,2.105000e+06,"(51.4541892, -0.1588555)",51.454189,-0.158856
1092,BELSIZE CRESCENT,2.000000e+06,"(51.5495675, -0.1727325)",51.549568,-0.172733
1135,BERESFORD TERRACE,2.100000e+06,"(55.9757427, -3.2031975)",55.975743,-3.203197
1193,BETTRIDGE ROAD,2.025000e+06,"(57.0320376, -2.1476985)",57.032038,-2.147699
1384,BLYTHS WHARF,2.000000e+06,"(51.50900105, -0.034583431747228985)",51.509001,-0.034583


In [54]:
df = df_affordable.drop(columns=['city_coord'])

In [55]:
df

,Street,Avg_Price,Latitude,Longitude
146,AIREDALE AVENUE,2.022500e+06,53.436368,-1.106305
197,ALBION STREET,2.096667e+06,-33.819759,151.010273
412,ANTHONY WAY,2.000000e+06,44.094651,-123.151242
554,ASHCHURCH PARK VILLAS,2.150000e+06,51.500051,-0.242173
673,AVENUE ROAD,2.143471e+06,51.406797,-0.049519
762,BALLINGDON ROAD,2.105000e+06,51.454189,-0.158856
1092,BELSIZE CRESCENT,2.000000e+06,51.549568,-0.172733
1135,BERESFORD TERRACE,2.100000e+06,55.975743,-3.203197
1193,BETTRIDGE ROAD,2.025000e+06,57.032038,-2.147699
1384,BLYTHS WHARF,2.000000e+06,51.509001,-0.034583


In [57]:
address = 'London, UK'

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [58]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [62]:
#Define Foursquare Credentials and Version

CLIENT_ID = 'KRTD1FJV3FCEIKJELRVOU5V4IMO1RQPFO0NK0GYZK52GBELH' # Foursquare ID
CLIENT_SECRET = 'JFFWFIJDN0RG2DHGS1G0UQH44WRDMZXBJHIWOAPI2F4EWGHK' # Foursquare Secret
VERSION = '20200730' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: KRTD1FJV3FCEIKJELRVOU5V4IMO1RQPFO0NK0GYZK52GBELH
CLIENT_SECRET:JFFWFIJDN0RG2DHGS1G0UQH44WRDMZXBJHIWOAPI2F4EWGHK


Now that everything is ready, I can proceed to the Modeling phase. I will analyse neighbourhoods so that I can recommend the most profitable investments for their needs. As mentioned in the Background Section, recommendations of profitable venues according to amenities and essential facilities such as schools, hospitals & supermarkets will be provided.

## 4. Modeling
I will use the k-means clustering technique will be used for this project as it is accurate, efficient and highly flexible.

In [63]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [64]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

AIREDALE AVENUE
ALBION STREET
ANTHONY WAY
ASHCHURCH PARK VILLAS
AVENUE ROAD
BALLINGDON ROAD
BELSIZE CRESCENT
BERESFORD TERRACE
BETTRIDGE ROAD
BLYTHS WHARF
BOSTON PLACE
BRACKENBURY GARDENS
BRAMSHOT AVENUE
BROADLANDS ROAD
BRONDESBURY PARK
BROOKFIELD PARK
BROWNING CLOSE
BRYANSTON SQUARE
BUNKERS HILL
BYWATER STREET
CALTON AVENUE
CANFIELD GARDENS
CARLISLE ROAD
CARLYLE CLOSE
CHOLMELEY CRESCENT
CLARE LAWN AVENUE
CLEVELAND SQUARE
COLINETTE ROAD
COLLEGE CROSS
COLVILLE PLACE
COTSWOLD MEWS
COVERDALE ROAD
CRANLEY MEWS
CUMBERLAND TERRACE
DENBIGH CLOSE
DONNE PLACE
DRYBURGH ROAD
DUNSTAN ROAD
ECCLESTON MEWS
EGERTON PLACE
ELSWORTHY ROAD
ESSEX PARK MEWS
EYNELLA ROAD
GLOUCESTER SQUARE
GRANARD ROAD
GREAT RUSSELL STREET
GREEN CLOSE
GROSVENOR GARDENS
GUION ROAD
HAMBLEDON PLACE
HANOVER YARD
HENDERSON ROAD
HENDON WOOD LANE
HEWER STREET
HIGHLEVER ROAD
HILLGATE PLACE
HOBHOUSE COURT
HOLLAND PARK
IVOR PLACE
KINGS GATE WALK
LAURIER ROAD
LEINSTER MEWS
LONG LANE
MANSON MEWS
MARGIN DRIVE
MAUNSEL STREET
MEADOWBANK
MEL

In [65]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,AIREDALE AVENUE,53.436368,-1.106305,Forget me not,53.435476,-1.110751,Antique Shop
1,AIREDALE AVENUE,53.436368,-1.106305,Scarbrough Arms,53.432736,-1.106993,Pub
2,AIREDALE AVENUE,53.436368,-1.106305,LloydsPharmacy,53.433125,-1.109704,Pharmacy
3,AIREDALE AVENUE,53.436368,-1.106305,Rocco's Italian Kitchen,53.432410,-1.109580,Italian Restaurant
4,ALBION STREET,-33.819759,151.010273,Dosa Hut,-33.821581,151.009332,Indian Restaurant
5,ALBION STREET,-33.819759,151.010273,Ginger Indian Restaurant,-33.820800,151.008850,Indian Restaurant
6,ALBION STREET,-33.819759,151.010273,Chatkazz,-33.821766,151.008440,Indian Restaurant
7,ALBION STREET,-33.819759,151.010273,Circa,-33.818687,151.005310,Café
8,ALBION STREET,-33.819759,151.010273,Not Just Curries,-33.821800,151.008760,Indian Restaurant
9,ALBION STREET,-33.819759,151.010273,Sweet Land Patisserie,-33.822240,151.009270,Café


In [66]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
AIREDALE AVENUE,4,4,4,4,4,4
ALBION STREET,12,12,12,12,12,12
ASHCHURCH PARK VILLAS,29,29,29,29,29,29
AVENUE ROAD,4,4,4,4,4,4
BALLINGDON ROAD,10,10,10,10,10,10
BELSIZE CRESCENT,13,13,13,13,13,13
BERESFORD TERRACE,5,5,5,5,5,5
BETTRIDGE ROAD,5,5,5,5,5,5
BLYTHS WHARF,17,17,17,17,17,17


In [67]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 289 uniques categories.


In [68]:
location_venues.shape

(2525, 7)

In [69]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,AIREDALE AVENUE,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AIREDALE AVENUE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AIREDALE AVENUE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AIREDALE AVENUE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBION STREET,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
GROUPED_LONDON = venues_onehot.groupby('Street').mean().reset_index()
GROUPED_LONDON

,Street,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,AIREDALE AVENUE,0.00,0.0,0.0,0.000000,0.000000,0.250000,0.00,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
1,ALBION STREET,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
2,ASHCHURCH PARK VILLAS,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.034483,0.000000,0.000000,0.0
3,AVENUE ROAD,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
4,BALLINGDON ROAD,0.00,0.0,0.0,0.000000,0.000000,0.100000,0.00,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.100000,0.100000,0.000000,0.0
5,BELSIZE CRESCENT,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
6,BERESFORD TERRACE,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
7,BETTRIDGE ROAD,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
8,BLYTHS WHARF,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
9,BOSTON PLACE,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0


In [71]:
GROUPED_LONDON.shape

(98, 290)

In [74]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in GROUPED_LONDON['Street']:
    print("----"+hood+"----")
    temp = GROUPED_LONDON[GROUPED_LONDON['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----AIREDALE AVENUE----
                venue  freq
0                 Pub  0.25
1        Antique Shop  0.25
2            Pharmacy  0.25
3  Italian Restaurant  0.25
4   Accessories Store  0.00


----ALBION STREET----
               venue  freq
0  Indian Restaurant  0.33
1               Café  0.17
2         Sports Bar  0.08
3        Coffee Shop  0.08
4               Park  0.08


----ASHCHURCH PARK VILLAS----
               venue  freq
0      Grocery Store  0.14
1               Park  0.10
2                Pub  0.10
3  Indian Restaurant  0.07
4             Bakery  0.07


----AVENUE ROAD----
               venue  freq
0      Grocery Store  0.25
1   Tapas Restaurant  0.25
2               Park  0.25
3       Tram Station  0.25
4  Accessories Store  0.00


----BALLINGDON ROAD----
                venue  freq
0                 Pub   0.3
1       Women's Store   0.1
2           Wine Shop   0.1
3        Antique Shop   0.1
4  Italian Restaurant   0.1


----BELSIZE CRESCENT----
                venue  

                 venue  freq
0                  Pub  0.13
1          Coffee Shop  0.07
2    French Restaurant  0.05
3   Mexican Restaurant  0.03
4  Arts & Crafts Store  0.03


----HENDERSON ROAD----
                    venue  freq
0          Clothing Store  0.24
1  Furniture / Home Store  0.06
2          Ice Cream Shop  0.06
3              Shoe Store  0.06
4             Snack Place  0.06


----HENDON WOOD LANE----
                venue  freq
0  Athletics & Sports   1.0
1   Accessories Store   0.0
2        Noodle House   0.0
3     Organic Grocery   0.0
4        Optical Shop   0.0


----HEWER STREET----
                       venue  freq
0                       Café  0.14
1                Pizza Place  0.10
2                     Bakery  0.10
3                       Park  0.05
4  Middle Eastern Restaurant  0.05


----HIGHLEVER ROAD----
               venue  freq
0               Park   0.2
1       Soccer Field   0.2
2  Convenience Store   0.1
3                Pub   0.1
4      Grocery Store 

               venue  freq
0               Bank   1.0
1  Accessories Store   0.0
2    Organic Grocery   0.0
3       Optical Shop   0.0
4        Opera House   0.0


----TIERNEY LANE----
               venue  freq
0     Sandwich Place   1.0
1  Accessories Store   0.0
2  Other Repair Shop   0.0
3       Optical Shop   0.0
4        Opera House   0.0


----TITE STREET----
                  venue  freq
0                   Pub  0.08
1           Pizza Place  0.05
2  Gym / Fitness Center  0.05
3   Japanese Restaurant  0.05
4           Coffee Shop  0.05


----TYSOE STREET----
         venue  freq
0          Pub  0.18
1  Coffee Shop  0.07
2         Café  0.05
3         Park  0.05
4          Bar  0.04


----WALBROOK----
                 venue  freq
0    Convenience Store  0.25
1  American Restaurant  0.25
2          Gas Station  0.25
3                 Bank  0.25
4           Playground  0.00


----WARWICK COURT----
                venue  freq
0   Convenience Store  0.17
1         Pizza Place  0.17
2

In [75]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [76]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [78]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = GROUPED_LONDON['Street']

for ind in np.arange(GROUPED_LONDON.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(GROUPED_LONDON.iloc[ind, :], num_top_venues)

In [79]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AIREDALE AVENUE,Italian Restaurant,Pharmacy,Antique Shop,Pub,Zoo,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit
1,ALBION STREET,Indian Restaurant,Café,Gym,Sports Bar,Coffee Shop,Dessert Shop,Pub,Park,Zoo,English Restaurant
2,ASHCHURCH PARK VILLAS,Grocery Store,Park,Pub,Mediterranean Restaurant,Coffee Shop,Fish & Chips Shop,Indian Restaurant,Bakery,Japanese Restaurant,Café
3,AVENUE ROAD,Tapas Restaurant,Grocery Store,Park,Tram Station,Zoo,Donut Shop,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space
4,BALLINGDON ROAD,Pub,Italian Restaurant,Women's Store,Wine Shop,Bakery,Coffee Shop,Antique Shop,Café,Zoo,English Restaurant


In [80]:
venues_sorted.shape

(98, 11)

In [81]:
GROUPED_LONDON.shape

(98, 290)

In [82]:
GROUPED_LONDON=df

Now that we can see the most profitable real estate investments in London, we can cluster properties by venues/facilities/amenities nearby.

In [85]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

GROUPED_LONDON_clustering = GROUPED_LONDON.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(GROUPED_LONDON_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([0, 2, 0, 1, 1, 2, 0, 2, 0, 0, 1, 1, 3, 4, 2, 1, 1, 3, 0, 0, 0, 3,
       3, 3, 0, 4, 1, 2, 3, 2, 1, 4, 1, 2, 4, 0, 1, 0, 2, 3, 4, 2, 0, 4,
       0, 1, 2, 2, 4, 2], dtype=int32)

In [110]:
#Dataframe to include Clusters

GROUPED_LONDON_clustering=df
GROUPED_LONDON_clustering.head()

,Street,Avg_Price,Latitude,Longitude
146,AIREDALE AVENUE,2.022500e+06,53.436368,-1.106305
197,ALBION STREET,2.096667e+06,-33.819759,151.010273
412,ANTHONY WAY,2.000000e+06,44.094651,-123.151242
554,ASHCHURCH PARK VILLAS,2.150000e+06,51.500051,-0.242173
673,AVENUE ROAD,2.143471e+06,51.406797,-0.049519


In [111]:
GROUPED_LONDON_clustering.shape

(110, 4)

In [112]:
df.shape

(110, 4)

In [113]:
GROUPED_LONDON_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [114]:
df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [115]:
# add clustering labels
GROUPED_LONDON_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
GROUPED_LONDON_clustering = GROUPED_LONDON_clustering.join(venues_sorted.set_index('Street'), on='Street')

GROUPED_LONDON_clustering.head(30) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
146,AIREDALE AVENUE,2.022500e+06,53.436368,-1.106305,0,Italian Restaurant,Pharmacy,Antique Shop,Pub,Zoo,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit
197,ALBION STREET,2.096667e+06,-33.819759,151.010273,2,Indian Restaurant,Café,Gym,Sports Bar,Coffee Shop,Dessert Shop,Pub,Park,Zoo,English Restaurant
412,ANTHONY WAY,2.000000e+06,44.094651,-123.151242,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
554,ASHCHURCH PARK VILLAS,2.150000e+06,51.500051,-0.242173,1,Grocery Store,Park,Pub,Mediterranean Restaurant,Coffee Shop,Fish & Chips Shop,Indian Restaurant,Bakery,Japanese Restaurant,Café
673,AVENUE ROAD,2.143471e+06,51.406797,-0.049519,1,Tapas Restaurant,Grocery Store,Park,Tram Station,Zoo,Donut Shop,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space
762,BALLINGDON ROAD,2.105000e+06,51.454189,-0.158856,2,Pub,Italian Restaurant,Women's Store,Wine Shop,Bakery,Coffee Shop,Antique Shop,Café,Zoo,English Restaurant
1092,BELSIZE CRESCENT,2.000000e+06,51.549568,-0.172733,0,Italian Restaurant,Movie Theater,Bakery,Bagel Shop,Greek Restaurant,Café,Pub,Hotel,Bed & Breakfast,Indian Restaurant
1135,BERESFORD TERRACE,2.100000e+06,55.975743,-3.203197,2,Bar,Cosmetics Shop,Pub,Bike Trail,Supermarket,Farmers Market,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit
1193,BETTRIDGE ROAD,2.025000e+06,57.032038,-2.147699,0,Bar,Grocery Store,Park,Chinese Restaurant,Dry Cleaner,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit
1384,BLYTHS WHARF,2.000000e+06,51.509001,-0.034583,0,Pub,Gym / Fitness Center,Italian Restaurant,Gastropub,Beer Garden,Restaurant,Trail,Turkish Restaurant,Park,Breakfast Spot


In [116]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(GROUPED_LONDON_clustering['Latitude'], GROUPED_LONDON_clustering['Longitude'], GROUPED_LONDON_clustering['Street'], GROUPED_LONDON_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [117]:
GROUPED_LONDON_clustering.loc[GROUPED_LONDON_clustering['Cluster Labels'] == 0, GROUPED_LONDON_clustering.columns[[1] + list(range(5, GROUPED_LONDON_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
146,2022500.0,Italian Restaurant,Pharmacy,Antique Shop,Pub,Zoo,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit
412,2000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1092,2000000.0,Italian Restaurant,Movie Theater,Bakery,Bagel Shop,Greek Restaurant,Café,Pub,Hotel,Bed & Breakfast,Indian Restaurant
1193,2025000.0,Bar,Grocery Store,Park,Chinese Restaurant,Dry Cleaner,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit
1384,2000000.0,Pub,Gym / Fitness Center,Italian Restaurant,Gastropub,Beer Garden,Restaurant,Trail,Turkish Restaurant,Park,Breakfast Spot


In [118]:
GROUPED_LONDON_clustering.loc[GROUPED_LONDON_clustering['Cluster Labels'] == 1, GROUPED_LONDON_clustering.columns[[1] + list(range(5, GROUPED_LONDON_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
554,2.150000e+06,Grocery Store,Park,Pub,Mediterranean Restaurant,Coffee Shop,Fish & Chips Shop,Indian Restaurant,Bakery,Japanese Restaurant,Café
673,2.143471e+06,Tapas Restaurant,Grocery Store,Park,Tram Station,Zoo,Donut Shop,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space
1435,2.167500e+06,Pub,Café,Coffee Shop,Pizza Place,Supermarket,Park,Gym,Bus Stop,Fish & Chips Shop,Canal Lock
1515,2.150000e+06,Grocery Store,Coffee Shop,Pub,Hotel,Street Food Gathering,Gym,Falafel Restaurant,Chinese Restaurant,Latin American Restaurant,Indian Restaurant
1775,2.150000e+06,Discount Store,Recreation Center,Home Service,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant


In [119]:
GROUPED_LONDON_clustering.loc[GROUPED_LONDON_clustering['Cluster Labels'] == 2, GROUPED_LONDON_clustering.columns[[1] + list(range(5, GROUPED_LONDON_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
197,2.096667e+06,Indian Restaurant,Café,Gym,Sports Bar,Coffee Shop,Dessert Shop,Pub,Park,Zoo,English Restaurant
762,2.105000e+06,Pub,Italian Restaurant,Women's Store,Wine Shop,Bakery,Coffee Shop,Antique Shop,Café,Zoo,English Restaurant
1135,2.100000e+06,Bar,Cosmetics Shop,Pub,Bike Trail,Supermarket,Farmers Market,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit
1751,2.090357e+06,Park,Japanese Restaurant,Coffee Shop,Pub,Farmers Market,Deli / Bodega,Gym / Fitness Center,Zoo,Electronics Store,English Restaurant
2989,2.124375e+06,Convenience Store,Coffee Shop,Cricket Ground,Café,Gastropub,Grocery Store,Tennis Court,Bus Stop,Gym / Fitness Center,Zoo


In [120]:
GROUPED_LONDON_clustering.loc[GROUPED_LONDON_clustering['Cluster Labels'] == 3, GROUPED_LONDON_clustering.columns[[1] + list(range(5, GROUPED_LONDON_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1559,2.177900e+06,Dance Studio,Spa,Bookstore,Gym,Farmers Market,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant
1867,2.197583e+06,Hotel,Sandwich Place,Coffee Shop,Lebanese Restaurant,Middle Eastern Restaurant,Pub,Bakery,Hotel Bar,Italian Restaurant,Persian Restaurant
2153,2.188333e+06,Café,Coffee Shop,Italian Restaurant,Pizza Place,Sandwich Place,Hotel,Grocery Store,Bus Stop,Bakery,Bookstore
2226,2.200000e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2242,2.175000e+06,Construction & Landscaping,Zoo,Farmers Market,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Fast Food Restaurant


In [121]:
GROUPED_LONDON_clustering.loc[GROUPED_LONDON_clustering['Cluster Labels'] == 4, GROUPED_LONDON_clustering.columns[[1] + list(range(5, GROUPED_LONDON_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1704,2043000.0,Harbor / Marina,Bed & Breakfast,Boat or Ferry,Pub,Farmers Market,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor
2789,2083000.0,Golf Course,Business Service,Park,Soccer Field,Eye Doctor,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit
3286,2065000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3762,2075000.0,Arts & Crafts Store,Pub,Park,Falafel Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor
4436,2078000.0,Hotel Bar,Pub,Hotel,Gym / Fitness Center,Department Store,Martial Arts Dojo,French Restaurant,Italian Restaurant,Grocery Store,Multiplex


## Results and Discussion

Even though, the UK has left the European Union it is still a place that both citizens and non-citizens would like to invest especially in London.

THE RESULTS 

First of all, West London (Kensington, Chelsea, Marylebone areas) and North-West London (Camden Town, Hampsted areas) were expected to pop as a profitable investment due to the wide range of amenities and essential facilitiesand venues. Yet South-West London (Brixton and Wandsworth, areas) and also Limehouse in East London are also arising as next future venues with a wide range of amenities and facilities. Therefore, South-West London and Limehouse could be a good investment opportunity for those who do not wish to invest in luxurious neighbourhoods like Kensington.

Secondly, by using the clusters the results can be analysed. In a city like London, there will be lots of pubs and restaurants but the clusters show different patterns. As you can see, the 3rd and 4th clusters tell us that those areas are more for people who are more active as it has an harbour, golf course, dance studios and fitness centres. The clusters 0, 1 and 2 can be interesting for investors who love going to pubs cafes and restaurants.

## Conclusion

The UK withdrawal from the European Union had affected the British housing market both for the  citizens and non-citizen buyers as the government made it more expensive for landlords and overseas buyers to purchase houses. Especially, real estate in London comes at quite a steep price. 
Therefore I tried to find the wisest investment options for people who seek a house in London now that Brexit is finalised. Recommendations of profitable venues according to amenities and essential facilities such as schools, hospitals & supermarkets was also a part of this project.

Data on London properties and the relative price paid data were extracted from the HM Land Registry. For data visialisation, FourSquare API interface is used. Then the collected data from HM Land Registry and Foursquare API is merged and showed the most profitable investments in London.

The Methodology for this project will start with 1. Collection of Data then continue with 2. Exploring and Understanding Data, 3. Data Preparation and Preprocessing and 4. Modelling. I will use the k-means clustering technique will be used for this project as it is accurate, efficient and highly flexible.

In conclusion, even though, the UK has left the European Union London is still a place that both people would like to invest. Although West London and North-West London were expected to pop as a profitable investment due to the wide range of amenities and essential facilitiesand venues. Yet South-West London (Brixton and Wandsworth, areas) and also Limehouse in East London are also arising as next future venues with a wide range of amenities and facilities. Therefore, South-West London and Limehouse could be a good investment opportunity for those who do not wish to invest in luxurious neighbourhoods like Kensington.
Secondly, by using the clusters the results can be analysed. In a city like London, there will be lots of pubs and restaurants but the clusters show different patterns. As you can see, the 3rd and 4th clusters tell us that those areas are more for people who are more active as it has an harbour, golf course, dance studios and fitness centres. The clusters 0, 1 and 2 can be interesting for investors who love going to pubs cafes and restaurants.